In [1]:
# First define the following parameters in this cell
# Then run the entire notebook

## filename of xml input
inputFilename = "pardok-wp18_berlin.xml"
#inputFilename = "Export_badenWürttemberg_WP16.xml"
#inputFilename = "exportWP6_brandenburg.xml"

## VTyp to take as process type
vtyp = "Antrag"
#vtyp = "Gesetz"

## output filename
outputFilename = './WP18_Berlin_' + vtyp + '.xes'
#outputFilename = './WP16_BadenWürttemberg_' + vtyp + '.xes'
#outputFilename = './WP6_Brandenburg_' + vtyp + '.xes'




In [2]:
# read data into dict
import pandas as pd
import xmltodict
import pprint
import pandas as pd
pp = pprint.PrettyPrinter(depth=4)

with open(inputFilename) as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [5]:
# get all Vorgaenge
vorgaenge = pd.DataFrame.from_dict(data_dict["Export"]["Vorgang"])

#print(vorgaenge.keys())
#print("\n")

groupedByType = vorgaenge.groupby(['VTyp'])
#print(groupedByType['VTyp'].count())

In [6]:
docsOfVorgaenge = groupedByType.get_group(vtyp)["Dokument"]
vSysLOfVorgange = groupedByType.get_group(vtyp)["VSysL"]
vSysOfVorgange = groupedByType.get_group(vtyp)["VSys"]
vorgangNebeneintraege = groupedByType.get_group(vtyp)["Nebeneintrag"]

# for a Vorgang there are several Deskriptoren stored in several Nebeneintraege
# -> get that information per Vorgang
vorgangDeskriptoren = []
for nebeneintraege in vorgangNebeneintraege:
    if type(nebeneintraege) is not list:
        if nebeneintraege is not dict:
            deskriptoren = []
        else:
            deskriptoren = [nebeneintraege.get("Desk", None)]
    else:
        deskriptoren = [obj.get("Desk", None) for obj in nebeneintraege]

    vorgangDeskriptoren.append([x for x in deskriptoren if x is not None])

allDocs = []

# add trace id to each document
# so that the documents can then be single events that belong to a specific trace in an event log

for idx, vorgang in enumerate(docsOfVorgaenge):
    helperX = vorgang
    if type(vorgang) is dict:
        helperX = [vorgang]
        
    for doc in helperX:
        # add case id column to each document
        # call it case:concept:name as this is the name for pm4py transformation into XES format
        doc['case:concept:name'] = idx

        doc['case:VSys'] = vSysOfVorgange.iloc[idx]
        doc['case:VSysL'] = vSysLOfVorgange.iloc[idx]
        doc['case:VorgangsDeskriptoren'] = vorgangDeskriptoren[idx]

        # for grouping reasons also sort all other entries that can be sorted
        doc_sortedEntries = {key: sorted(value) if isinstance(value, list) else value for key, value in doc.items()}
        # now add this doc as a row 
        allDocs.append(doc_sortedEntries)


# turn docs into data frame, so each row is a document now
df = pd.DataFrame(allDocs)
#print(df.keys())

# turn date string into date time object
df["DokDat"] = pd.to_datetime(df['DokDat'], format='%d.%m.%Y')
#print(df["DokDat"])

In [8]:
# use pm4py to create XES file
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

#eventlog = df.copy(deep=True)
#eventlog.rename(columns={'DokDat': 'time:timestamp', 'DokTypL': 'concept:name'}, inplace=True)

event_log = pm4py.format_dataframe(df, case_id='case:concept:name', activity_key='DokTypL', timestamp_key='DokDat')
#start_activities = pm4py.get_start_activities(event_log)
#end_activities = pm4py.get_end_activities(event_log)
#print("Start activities: {}\nEnd activities: {}".format(start_activities, end_activities))

pm4py.write_xes(event_log, outputFilename)

/home/phillmann/mambaforge/lib/python3.9/site-packages/pm4py/utils.py:96: UserWarning: Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.
  warnings.warn("Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.")
/home/phillmann/mambaforge/lib/python3.9/site-packages/pm4py/utils.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[constants.CASE_CONCEPT_NAME] = df[constants.CASE_CONCEPT_NAME].astype("string")
/home/phillmann/mambaforge/lib/python3.9/site-packages/pm4py/utils.py:101: SettingWithCopyWarning: 
A value is 

exporting log, completed traces ::   0%|          | 0/1248 [00:00<?, ?it/s]